We have an excel databases where I control payments made by clients and who has paid and who has due invoices. I want to send automatic emails charging the due invoices from a certain date.

""" 1 - Installing libraries we will use in our program. We need
pip install pywin32 (to allow us to program automatic emails through outlook).
pip install panda (to use data analysis functions in python).
pip install date time (to use date functions in python).

This must be done in the terminal only the first time.
"""

In [32]:
# 2 - Importing the libraries to this file

import win32com.client as client
import pandas as pd
import datetime as dt

In [ ]:
# 3 - Reading and displaying our excel databases.
table = pd.read_excel('payments.xlsx')
display(table)

,CPF,Valor em aberto,Data Prevista para pagamento,Status,E-mail,NF
0,44204824413,1195,2022-01-23,Pago,meuclientedevedor@gmail.com,197
1,27333842934,1670,2022-01-26,Em aberto,meuclientedevedor+1@gmail.com,177
2,27337412934,1345,2022-01-12,Em aberto,meuclientedevedor+2@gmail.com,133
3,53713111219,1372,2022-01-24,Em aberto,meuclientedevedor+3@gmail.com,155
4,17537122867,1718,2022-01-31,Pago,meuclientedevedor+4@gmail.com,149
...,...,...,...,...,...,...
195,26130190204,1060,2022-01-19,Pago,meuclientedevedor+195@gmail.com,134
196,50695164206,1587,2022-01-21,Pago,meuclientedevedor+196@gmail.com,117
197,68885370313,1842,2022-01-15,Pago,meuclientedevedor+197@gmail.com,107
198,58701953352,1604,2022-01-20,Pago,meuclientedevedor+198@gmail.com,116


In [34]:
# 4 - Getting informations about the type of our objects - we need to now what kind are our variables
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   CPF                           200 non-null    int64         
 1   Valor em aberto               200 non-null    int64         
 2   Data Prevista para pagamento  200 non-null    datetime64[ns]
 3   Status                        200 non-null    object        
 4   E-mail                        200 non-null    object        
 5   NF                            200 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 9.5+ KB


In [35]:
# 5- Verifying which date is today using the function now from the module datetime of the datetime library.
today = dt.datetime.now()
print(today)

2025-10-03 11:06:20.466363


In [ ]:
# 6 - Colecting data only from customers who have due invoices according to due date.
table_dueinvoices = table.loc[
    (table['Status'] == 'Em aberto') & 
    (table['Data Prevista para pagamento'] < today)]

display(table_dueinvoices)

,CPF,Valor em aberto,Data Prevista para pagamento,Status,E-mail,NF
1,27333842934,1670,2022-01-26,Em aberto,meuclientedevedor+1@gmail.com,177
2,27337412934,1345,2022-01-12,Em aberto,meuclientedevedor+2@gmail.com,133
3,53713111219,1372,2022-01-24,Em aberto,meuclientedevedor+3@gmail.com,155
5,54769185414,1533,2022-01-13,Em aberto,meuclientedevedor+5@gmail.com,118
6,44744009510,1411,2022-01-12,Em aberto,meuclientedevedor+6@gmail.com,191
...,...,...,...,...,...,...
190,71953312562,1657,2022-01-12,Em aberto,meuclientedevedor+190@gmail.com,178
191,62203137527,1419,2022-01-16,Em aberto,meuclientedevedor+191@gmail.com,200
192,34219196712,1018,2022-01-28,Em aberto,meuclientedevedor+192@gmail.com,171
193,26316453421,1487,2022-01-28,Em aberto,meuclientedevedor+193@gmail.com,129


In [ ]:
# 7 - How to send an automatic email through outlook

# Creation of an object to open outlook aplication
outlook = client.Dispatch('Outlook.Application')

# In case you have more that one account associated with your outlook, we have to determine from which one the email is going to be sent.
sender = outlook.session.Accounts['50049839@europeia.pt']

""" Creation of the email;we have to use fuction for to repeat the procedure to each email box; we need to create a list to include in the body
of the email each item of our table we intend to include
"""
## Creation of the a variable where we use the table due invoices and say to python which columns we want to include in the list.

data = table_dueinvoices[['Valor em aberto', 'Data Prevista para pagamento', 'E-mail', 'NF']].values.tolist()

for item in data:
    recipient = item[2]
    nf = item[3]
    due_date = item[1]
    due_date = due_date.strftime("%d/%m/%Y") #method of datetime to bring this data from datetime model and put it in string model
    amount = item[0]

# Creation and sending of the email.

message = outlook.CreateItem(0)
message.display()
message.To = recipient
message.Subject = 'Atraso no pagamento'
message.Body = f'''

Dear Customer,

We kindly remind you about the payment of the invoice with number {nf} with due date {due_date} of the amount of Eur {amount:.2f}.

Kind regards,
Joseph Potter
'''

message._oleobj_.Invoke(*(64209,0,8,0,sender)) #Define wo the sende is.
message.Save() #Saving the email.
message.Send() #Sending the email.